# 1. Convolution Neural Network
A Convolutional Neural Network (CNN) is a type of deep learning model designed to process data with a grid-like topology, such as images. It is widely used in image classification, object detection, and video analysis.

## Component

### Convolution Layer
Applies filters (kernels) to the input to extract local features like edges, corners, or textures.
```Py
torch.nn.Conv2d(
    in_channels,
    out_channels,
    kernel_size,
    stride=1,
    padding=0,
    ...
)
```
https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D

### Pooling Layer
Reduces the spatial size of the feature maps to lower computational cost and retain important information.

ex) Max Pooling, Average Pooling

```Py
torch.nn.MaxPool2d(
    kernel_size,
    stride=None,
    padding=0
    ...
)
```
https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPooling2D

### Relu Activation Function
Introduces non-linearity by setting negative values to zero and keeping positive values.
```Py
torch.nn.functional.relu(
    input
)
```

### Fully Connected Layer
Typically at the end of the network, responsible for making final decisions (like classification) based on the extracted features.

```Py
torch.nn.Linear(
    in_features,
    out_features,
    ...
)
```
https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense

# 2. Import Libaray

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# 3. MNIST data
The MNIST (Mixed National Institute of Standards and Technology) dataset
is a collection of handwritten digit images, ranging from 0 to 9.

## Detail
Image Size: 28 × 28 pixels, grayscale

Data Split: Training images: 60,000, Test images: 10,000

Number of Classes: 10 (digits from 0 to 9)

<img src="https://storage.googleapis.com/tfds-data/visualization/fig/mnist-3.0.1.png" width="500"/>

In [6]:
transform = transforms.Compose([
    transforms.ToTensor(),  # [0,1]
    transforms.Normalize((0.5,), (0.5,))  # [-1,1]
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



# 4. Model Implement
<img src="./그림1.jpg" width="500"/>
<img src="./그림2.jpg" width="500"/>


In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(64 * 5 * 5, 10)  # MNIST는 28x28 → 14x14 → 7x7

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [9]:
for epoch in range(15):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}")

Epoch 1, Loss: 471.1856
Epoch 2, Loss: 135.1770
Epoch 3, Loss: 95.7337
Epoch 4, Loss: 76.2303
Epoch 5, Loss: 64.5951
Epoch 6, Loss: 56.4604
Epoch 7, Loss: 51.0703
Epoch 8, Loss: 46.9305
Epoch 9, Loss: 43.6898
Epoch 10, Loss: 41.0525
Epoch 11, Loss: 38.9959
Epoch 12, Loss: 36.9675
Epoch 13, Loss: 35.3647
Epoch 14, Loss: 34.0223
Epoch 15, Loss: 32.7133


In [10]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9829


In [ ]:
from torchsummary import summary
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
         MaxPool2d-2           [-1, 32, 13, 13]               0
            Conv2d-3           [-1, 64, 11, 11]          18,496
         MaxPool2d-4             [-1, 64, 5, 5]               0
           Flatten-5                 [-1, 1600]               0
            Linear-6                   [-1, 10]          16,010
Total params: 34,826
Trainable params: 34,826
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.29
Params size (MB): 0.13
Estimated Total Size (MB): 0.43
----------------------------------------------------------------
